In [14]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as sm
from sklearn.metrics import mean_squared_error,r2_score,explained_variance_score
from datetime import date
import matplotlib.pyplot as plt
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="p0o9i8u7",
  database="jockey"
)

#features=['draw','addedWeight','declaredHorseWeight','horse_winPer',
#                 'horse_winCount','jockey_winPer','jockey_winCount','horse_newDistance','horse_newHorse',
#         'horse_last4SpeedRate','horse_latestSpeedRate','Days_from_lastRace']
#target=['finishTime']

features=['draw','horse_winPer',
                 'jockey_winPer','horse_newDistance','horse_newHorse',
         'horse_last4SpeedRate','horse_latestSpeedRate','Days_from_lastRace']
target_fea='finishTime'

target=[target_fea]
sql_history = 'select raceDate,raceSeqOfDay,distance,raceClass, horseId, place,lbs,'
for aFeature in features:
    sql_history = sql_history + aFeature +", "
sql_history = sql_history+ target_fea+ " from racecard"
dataset0 = pd.read_sql(sql_history,mydb)
#print(dataset0)
#Get new race data
sql_predict = 'select raceDate,raceSeqOfDay,distance,raceClass, horseId,'
for aFeature in features:
    sql_predict = sql_predict + aFeature +", "
sql_predict = sql_predict+ "rating from newrace"

dataset1 = pd.read_sql(sql_predict,mydb)
print(dataset1)


tableName="newrace"


#dataset0 = pd.read_csv(file0)
#dataset1 = pd.read_csv(file1)

    raceDate  raceSeqOfDay  distance  raceClass horseId  draw  horse_winPer  \
0   20190217             1      2000          4    V407     4          0.33   
1   20190217             1      2000          4    V192     1          0.13   
2   20190217             1      2000          4    B137     8          0.34   
3   20190217             1      2000          4    A325     2          0.22   
4   20190217             1      2000          4    A284     6          0.28   
5   20190217             1      2000          4    S015     5          0.21   
6   20190217             1      2000          4    V039     3          0.25   
7   20190217             1      2000          4    S147    10          0.31   
8   20190217             1      2000          4    A358     7          0.09   
9   20190217             1      2000          4    V369     9          0.53   
10  20190217             2      1000          4    B214     6          0.23   
11  20190217             2      1000          4    C

In [15]:
allDates = dataset1.raceDate.unique()
allDates.sort()
for raceDate in allDates:
    #if(raceDate <20180218):
    #     continue
    print(raceDate)
    dataset_ofADay = dataset1[(dataset1['raceDate']==raceDate)]
    allSeqs = dataset_ofADay.raceSeqOfDay.unique()
    for seq in allSeqs:
        dataset_test =  dataset_ofADay[(dataset1['raceSeqOfDay']==seq)]
        distance = dataset_test['distance'].iloc[0]
        raceClass = dataset_test['raceClass'].iloc[0]
        print(seq,distance,raceClass)
        
        dataset_train = dataset0[(dataset0['raceClass']==raceClass) & (dataset0['distance']==distance) 
                        & (dataset0['raceDate']!=raceDate)]
        #dataset_train.shape
        if(dataset_train.size==0):
            print("Empty History")
            continue
        #regression model of lbw or finishTime
        X_train=dataset_train[features]
        y_train=dataset_train[target] #lbw

        X_test=dataset_test[features]
        reg = LinearRegression()
        reg.fit(X_train,y_train)
        y_test_predicted=reg.predict(X_test)
        
        #plt.scatter(y_test, y_test_predicted,  color='black')
        #plt.show()
        #print("Mean squared error: %.2f"
        #      % mean_squared_error(y_test, y_test_predicted))
        # Explained variance score: 1 is perfect prediction
        #print('Variance score: %.2f' % r2_score(y_test, y_test_predicted))

        #explained_variance_score(y_test, y_test_predicted)
        df = pd.DataFrame(data=y_test_predicted, index=X_test.index,columns=['finishTime_predicted'])
        dataset_test_toshow=dataset_test[['raceDate','raceSeqOfDay','distance','raceClass','horseId']]
        dataset_test_withPredicted=dataset_test_toshow.merge(df,how='inner',left_index=True,right_index=True)
        dataset_test_withPredicted = dataset_test_withPredicted.sort_values(['finishTime_predicted'])
        #dataset_test_withPredicted

        predicted_place=1
        for index, row in dataset_test_withPredicted.iterrows():
            mycursor = mydb.cursor()
            raceDate_str = str(row['raceDate'])
            raceSeqOfDate =str(row['raceSeqOfDay'])
            horseId = str(row['horseId'])
            finishTime_predicted =str(row['finishTime_predicted'])
            ##print("Andrea:",raceDate_str,raceSeqOfDate,horseId,finishTime_predicted)
            sql = ("UPDATE "+tableName+ " SET predicted_place="+ str(predicted_place) +", predicted_finishTime = "+ finishTime_predicted +
            " WHERE raceDate = "+ raceDate_str + " and raceSeqOfDay = "+raceSeqOfDate + " and horseId ='"+horseId+ "' ")
            
            print(predicted_place)
            predicted_place =predicted_place+1
            #print(sql)
            mycursor.execute(sql)
            mydb.commit()
            print(mycursor.rowcount, " record(s) updated" ,row['raceSeqOfDay'] ,",", row['horseId'] )

20190217
1 2000 4
1
1  record(s) updated 1 , V407
2
1  record(s) updated 1 , V192
3
1  record(s) updated 1 , B137
4
1  record(s) updated 1 , S015
5
1  record(s) updated 1 , S147
6
1  record(s) updated 1 , V039
7
1  record(s) updated 1 , A325
8
1  record(s) updated 1 , A358
9
1  record(s) updated 1 , A284
10
1  record(s) updated 1 , V369
2 1000 4
1
1  record(s) updated 2 , C229
2
1  record(s) updated 2 , C224
3
1  record(s) updated 2 , A130
4
1  record(s) updated 2 , C226
5
1  record(s) updated 2 , B367
6
1  record(s) updated 2 , C105
7
1  record(s) updated 2 , B214
8
1  record(s) updated 2 , C130
9
1  record(s) updated 2 , C034
10
1  record(s) updated 2 , C086
11
1  record(s) updated 2 , C131
12
1  record(s) updated 2 , B177
13
1  record(s) updated 2 , T392
14
1  record(s) updated 2 , C134
3 1400 4
1
1  record(s) updated 3 , A088
2
1  record(s) updated 3 , B225
3
1  record(s) updated 3 , C007
4
1  record(s) updated 3 , B460
5
1  record(s) updated 3 , A125
6
1  record(s) updated 3 , A05